In [ ]:
from aocd.models import Puzzle
from typing import Tuple, Dict
from collections import defaultdict
from itertools import combinations, product, count, chain
from functools import lru_cache, reduce
from operator import mul
import re

puzzle = Puzzle(year=2020, day=16)
print(puzzle.input_data)

In [ ]:
re_rules = re.compile(r'(\w+\s?\w+): (\d+)-(\d+) or (\d+)-(\d+)')
sections = puzzle.input_data.split('\n\n')
rules = {}
for line in sections[0].splitlines():
    a,b,c,d,e = re_rules.match(line).groups()
    rules[a] = [(int(b),int(c)), (int(d),int(e))]
yours = [int(x) for x in sections[1].splitlines()[1].split(',')]
nearby = []
for line in sections[2].splitlines()[1:]:
    nearby.append([int(x) for x in line.split(',')])



In [ ]:
 # Part 1
invalids = []
for elem in chain.from_iterable(nearby):
    invalid = (not (rule[0][0] <= elem <= rule[0][1] or rule[1][0] <= elem <= rule[1][1]) for rule in                        rules.values())
    if all(invalid):
        invalids.append(elem)

answer_a = sum(invalids)
print(answer_a)

In [ ]:
puzzle.answer_a = answer_a